In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import datetime
from tqdm.auto import tqdm
from collections import Counter
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
# %load_ext line_profiler

This is my submission to the Kaggle, data science bowl. All code here is under the open source license referred to in the Kaggle competition rules.


## Inspect the data

In [ ]:
#train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv') #df.head()

In [ ]:
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv') #df.head()

In [ ]:
#print('Training.csv file has {} rows and {} columns'.format(train.shape[0], train.shape[1]))

In [ ]:
print('Test.csv file has {} rows and {} columns'.format(test.shape[0], test.shape[1]))

In [ ]:
#specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
#print('specs.csv file has {} rows and {} columns'.format(specs.shape[0], specs.shape[1]))

In [ ]:
#train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
#print('train_lablels.csv file has {} rows and {} columns'.format(train_labels.shape[0], train_labels.shape[1]))
#train_labels

In [ ]:
#sample = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')
#print('sample submission file has {} rows and {} columns'.format(sample.shape[0], sample.shape[1]))

## EDA

let's see how many installation_ids there are

In [ ]:
#users = train.installation_id.unique()
#len(users) # number of distinct users

There are 17000 unique installation_id's in the training set. Let's examine the breakdown of event types for an example `installation_id` (from the first row in the dataset)

In [ ]:
#types = train.loc[train['installation_id']=='0001e90f'].groupby(['type'],as_index=False).size().reset_index(name='counts')
#types

Below i calculate the total time spent on each different types of events in minutes for each installation_id. When I first tried to do this, I called train.loc without narrowing to the user data... a serious mistake from a code design point of view

## Ground truth

Note that the Bird Measurer assessment has two parts however we consider the it to be passed correctly if just the first part is passed, that is the 4110 code. Here, I duplicate the train_labels that were provided by parsing the train data set and calculating the accuracy. 

In [ ]:
#assessments = train.loc[(train['type'] == 'Assessment') & (((train['event_code'] == 4100)&(train['title'] != 'Bird Measurer (Assessment)')) | (train['event_code'] == 4110))].copy()
#assessments['num_correct'] = True
#assessments
test_assessments = test.loc[(test['type'] == 'Assessment') & (((test['event_code'] == 4100)&(test['title'] != 'Bird Measurer (Assessment)')) | (test['event_code'] == 4110))].copy()
test_assessments_for_union = test_assessments.copy()
test_assessments['num_correct'] = True

In [ ]:
test_assessments.shape

In [ ]:
#assessments.loc[assessments['event_data'].str.contains(":false,"),'num_correct'] = False

test_assessments.loc[test_assessments['event_data'].str.contains(":false,"),'num_correct'] = False

#assessments['num_incorrect'] = np.where(assessments.num_correct > 0,0,1)

test_assessments['num_incorrect'] = np.where(test_assessments.num_correct > 0,0,1)

The way I calculate the number of incorrect assessments is using a groupby(). However, we need to know both number incorrect as well as if there was a correct answer in separate columns. I chose two separately create these two columns and then merge them using an inner join, however this is probably not best practice, there must be a way to create two aggregate columns in a single step without creating two dataframes (stackoverflow this) **possible solution** using agg() allows for multiple aggregations - might do the trick

In [ ]:
#g = assessments.groupby(['game_session','installation_id','title'],as_index=False)['num_correct'].sum().sort_values(by=['installation_id'])
#h = assessments.groupby(['game_session','installation_id','title'],as_index=False)['num_incorrect'].sum().sort_values(by=['installation_id'])

ggg = test_assessments.groupby(['game_session','installation_id','title'],as_index=False)['num_correct'].sum().sort_values(by=['installation_id'])
hhh = test_assessments.groupby(['game_session','installation_id','title'],as_index=False)['num_incorrect'].sum().sort_values(by=['installation_id'])

In [ ]:
#g['num_correct'] = g['num_correct'].astype(int)      #g.shape
#h['num_incorrect'] = h['num_incorrect'].astype(int)   #h.shape

ggg['num_correct'] = ggg['num_correct'].astype(int)      #g.shape
hhh['num_incorrect'] = hhh['num_incorrect'].astype(int)   #h.shape

In [ ]:
#merged_inner = pd.merge(left=g,right=h, left_on='game_session', right_on='game_session')

test_merged_inner = pd.merge(left=ggg,right=hhh, left_on='game_session', right_on='game_session')

In [ ]:
#df1 = merged_inner[['game_session','installation_id_x','title_x','num_correct','num_incorrect']]
df2 = test_merged_inner[['game_session','installation_id_x','title_x','num_correct','num_incorrect']] 
pd.options.mode.chained_assignment = None
#df1.rename(columns = {'installation_id_x':'installation_id'}, inplace = True)
#df1.rename(columns = {'title_x':'title'}, inplace = True)
df2.rename(columns = {'installation_id_x':'installation_id'}, inplace = True)
df2.rename(columns = {'title_x':'title'}, inplace = True)

In [ ]:
#df1['accuracy'] = df1.apply(lambda row: row.num_correct/(row.num_correct + row.num_incorrect), axis=1)
df2['accuracy'] = df2.apply(lambda row: row.num_correct/(row.num_correct + row.num_incorrect), axis=1)

In [ ]:
# define accuracy groups

def groupacc(row):
    if row['num_correct'] == 1 and row['num_incorrect'] == 0:
        return 3
    if row['num_correct'] == 1 and row['num_incorrect'] == 1:
        return 2
    if row['num_correct'] == 1 and row['num_incorrect'] > 1:
        return 1
    if row['num_correct'] == 0: 
        return 0
    return 'Other'

#df1['accuracy_group'] = df1.apply(groupacc, axis=1)
df2['accuracy_group'] = df2.apply(groupacc, axis=1)

At this point I want to compare to see if the ground truths are the same as the one provided. I check the values below and they are all pretty much same except for a few unimportant trailing float differences.

In [ ]:
df2.shape

In [ ]:
#c = df1.sort_values(by=['game_session'])
#k = train_labels.sort_values(by=['game_session'])


In [ ]:
#comparison_array = c.values == k.values
#
#if False in comparison_array:
#    print ("Not the same")

In [ ]:
#len(np.where(comparison_array==False)[0])  # 46 differences due to floating point stuff

## Features 

$Y$ is the accuracy group for an assessment, e.g. this is what you are trying to predict.

$X$ will be a feature matrix for each such session collected by looking up data for that user, you only want to consider data 
up to the timestamp of the assessment you are training or predicting on.


There will also be features like "title" which tells you which one it is, this is not historical.

Let's pair down the train set to only the ids that actually took (e.g. started 2000) an assessment and also made an attempt (4100/4110) and do some more inspection, with comments detailing below.

one issue is features which have NA for some data - (accumulated previous accuracy group) -> our approach will be to substitue the mean value for the type.

**NOTE: YOU HAVE TO BUILD FEATURES FOR THE TEST SET AS WELL** 

below i do some encodings of some text titles to integers

In [ ]:
# some encodings
list_of_assessment_titles = list(set(test_assessments['title'].unique()))
list_of_assessment_titles.sort()

In [ ]:
assessment_titles_map = dict(zip(list_of_assessment_titles, np.arange(len(list_of_assessment_titles))))
assessment_titles_map

In [ ]:
#assessments['title'] = assessments['title'].map(assessment_titles_map)    # note if you run this twice it will break
#train_labels['title'] = train_labels['title'].map(assessment_titles_map)  # note if you run this twice it will break

In [ ]:
#print(len(assessments['game_session'].unique()))  # unique sessions of type assesment in train set
#train_reduced_users = assessments['installation_id'].unique()
#print(len(train_reduced_users))  # unique users who made at least one attempt on an assessment in train

In [ ]:
#train = train.loc[train['installation_id'].isin(train_reduced_users)]  # drop some data
#train.shape

In [ ]:
test_users = test.installation_id.unique()    # installation_ids in test set
print(len(test_users))
#len(set(test_users).intersection(set(users)))  # there is no intersection in installation_ids between train and test

At this point after inspection, it becomes clear that we need to predict the score for the last assessment in the test set. Also, despite some unclear language in the instructions, after some effort it became clear to me from inspection that **the last assessment (by timestamp) has been truncated for each user in the test set**. Some of the test set users have no previous assessments to learn from. 

To build features let's build some **accumulations**. We want accumulated accuracy groups on previous assessments. Possibly broken down for each assessment. Let's start there. Later we also want counts of time spent previously on various stuff and counts of events for previous stuff.

In [ ]:
mean_group_accs = [1.32,2.3,2.307,0.838,2.214]

**remember to improve these means by adding rows from the test set or external data would help also!!**

In [ ]:
list(mean_group_accs)

You need to combine the train data with the test data (except for last line)

In [ ]:
# code to combine train and test datas

test_predict = test.loc[test['type'] == 'Assessment'].groupby('installation_id',as_index=False).last()

# you need to combine test_assessments so that it has those last rows basically do a union at this stage

test_final_assessments = pd.concat([test_assessments_for_union,test_predict],sort=False)
test_final_assessments = test_final_assessments.sort_values(by=['installation_id','timestamp'])
#test_final_assessments

In [ ]:
def build_features(num,test_or_train):

    N=num   # to go to production set N = len(train_reduced_users)

    features = []   # this will by the full data for each assessment   X  & Y

    build_test = test_or_train   # False

    for j in tqdm(range(N)):

        if build_test == False:
            kklkklk =1
            #user = train_reduced_users[j]   
            #prev_hist = train.loc[train['installation_id']==user]   # for previous history counts, narrowed to user    
            #user_assessments = assessments.loc[assessments['installation_id']==user]  # for previous cum_accuracy_grps    
            #user_sessions = user_assessments['game_session'].unique()  # time ordered
        else:
            user = test_users[j]
            prev_hist = test.loc[test['installation_id']==user]   # for previous history counts, narrowed to user 
            user_sessions = test_final_assessments.loc[test_final_assessments['installation_id']==user]['game_session'].unique()

        counters = np.array([0.0,0.0,0.0,0.0,0.0])  # number of each type of assessment initialization

        cumalitive_acc_groups = np.array(list(mean_group_accs))   # initialization

        #cumalitive_acc_groups = np.array([0.0,0.0,0.0,0.0,0.0])   # alternative zero initialization

        magmapeak_counts = 0
        treetopcity_counts = 0
        crystalcaves_counts = 0

        game_counts = 0
        clip_counts = 0
        activity_counts = 0
        assess_counts = 0

        ii = 1

        for session in user_sessions:     # these are already time-ordered 

            # get the time at the start of the game_session

            # code to convert timestamp data to datetime data

            prev_hist['timestamp'] = pd.to_datetime(prev_hist['timestamp'])

            the_time = prev_hist.loc[prev_hist['game_session']==session]['timestamp'].iloc[0]

            the_past = prev_hist.loc[prev_hist['timestamp'] < the_time]  # this could be empty!! 

            num_events = the_past.groupby('world',as_index=False).size().reset_index(name='counts')

            num_sessions_by_type = the_past.groupby('game_session',as_index=False).last().groupby('type').size().reset_index(name='tcounts')

            for index, row in num_sessions_by_type.iterrows():
                if row['type'] == 'Game':
                    game_counts = row['tcounts']
                if row['type'] == 'Activity':
                    activity_counts = row['tcounts']
                if row['type'] == 'Assessment':
                    assess_counts = row['tcounts'] 
                if row['type'] == 'Clip':
                    clip_counts = row['tcounts']

            # now you want to count number of previous events for this user before this timestamp

            user_features = []

            user_features.append(clip_counts)
            user_features.append(activity_counts)
            user_features.append(assess_counts)
            user_features.append(game_counts)  

            if build_test == False:
                sesh_dat = train_labels.loc[train_labels['game_session'] == session]
                score = sesh_dat['accuracy_group'].iloc[0]
            else:
                if ii < len(user_sessions):
                    sesh_dat = df2.loc[df2['game_session'] == session]
                    score = sesh_dat['accuracy_group'].iloc[0]
                else:
                    score = 0         

            #title = sesh_dat['title'].iloc[0]

            title = prev_hist.loc[prev_hist['game_session']==session]['title'].iloc[0]
            
            title = assessment_titles_map[title]

            #user_features.append(user)

            user_features.append(title)

            total_accum = np.sum(cumalitive_acc_groups)/(np.sum(counters)+5.0)
            user_features.append(cumalitive_acc_groups[0]/(counters[0]+1.0))
            user_features.append(cumalitive_acc_groups[1]/(counters[1]+1.0))
            user_features.append(cumalitive_acc_groups[2]/(counters[2]+1.0))
            user_features.append(cumalitive_acc_groups[3]/(counters[3]+1.0))
            user_features.append(cumalitive_acc_groups[4]/(counters[4]+1.0))

            if title == 0:
                counters[0] += 1
                cumalitive_acc_groups[0] = cumalitive_acc_groups[0] + score
            elif title == 1:
                counters[1] += 1
                cumalitive_acc_groups[1] = cumalitive_acc_groups[1] + score
            elif title == 2:
                counters[2] += 1
                cumalitive_acc_groups[2] = cumalitive_acc_groups[2] + score
            elif title == 3:
                counters[3] += 1
                cumalitive_acc_groups[3] = cumalitive_acc_groups[3] + score
            elif title == 4:
                counters[4] += 1
                cumalitive_acc_groups[4] = cumalitive_acc_groups[4] + score

            user_features.append(total_accum)

            user_features.append(score)

            for index, row in num_events.iterrows():
                if row['world'] == 'MAGMAPEAK':
                    magmapeak_counts = row['counts']
                if row['world'] == 'TREETOPCITY':
                    treetopcity_counts = row['counts']
                if row['world'] == 'CRYSTALCAVES':
                    crystalcaves_counts = row['counts']  

            user_features.append(magmapeak_counts)
            user_features.append(treetopcity_counts)
            user_features.append(crystalcaves_counts)
            user_features.append(sum(num_events['counts']) - (magmapeak_counts + treetopcity_counts+crystalcaves_counts))
            user_features.append(sum(num_events['counts']))
            
            if build_test == False:
                features.append(user_features)
            else:
                if ii == len(user_sessions):
                    features.append(user_features)
            ii += 1
            
    return features

In [ ]:
test = pd.DataFrame(build_features(len(test_users),True))
test = test.round(2)
#test

In [ ]:
#train = pd.DataFrame(build_features(3614,False))
#train = train.round(2)
#train

In [ ]:
#train.to_csv('train.csv', index=False)

In [ ]:
train = pd.read_csv('/kaggle/input/traincsv/train.csv')

#to determine the kaggle kernel paths do
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

you can just upload the csv of train data instead of building it in the kaggle kernel!

At this point there is alot of **optimization, remove inefficiencies** use Counter() etc. that needs to be done to speed up the `build_features` function - you can search through fewer events.

Also, alot more features could be included such as some **durations**... alot more could be done here but for now we just want to go ahead and get it to work and then we can come back to feature engineering later if we have time

**another idea is to include the historical test data into train**


## Approach to modelling


Our approach to modelling is to work from simple models to complex models and to learn about how to implement each model correctly while building cross-validation into the pipeline as well. The idea for this competition is to learn how to build models, not neccessarily to win this competition... though that would be nice...

1. Simple regression (or logistic regression)

2. Single decision tree

3. KNN classifier 

4. Naive Bayes

5. Neural Network (probably will not score great but you should still implement)

6. 

The idea is to start with a **simple model** and **submit** and see how you improve as you choose different models!!
For this purpose a common framework, wrapper, for models does make sense.
**categoricals** the title - column 4 is a categorical, as is the target variable column 11



## Model eval - quadratic weighted kappa (qwk)

In [ ]:
def qwk(a1,a2):
    N = 4
    o = confusion_matrix(a1,a2)
    w = np.zeros(shape=(N,N))
    for i in range(N):
        for j in range(N):
            w[i,j] = (i-j)**2
    w = w/((N-1)*(N-1)) 
    w = w.round(3)
    
    e = np.outer(np.histogram(a1, [i for i in range(N+1)])[0],np.histogram(a2,[i for i in range(N+1)])[0])

    e = e/np.sum(e)
    o = o/np.sum(o)

    return 1 - np.sum(np.multiply(o,w))/np.sum(np.multiply(e,w))


In [ ]:
# preprocessing 

y_train = train['11']
y_test = test[11]
X_train = train.drop('11', axis=1)
X_test = test.drop(11, axis=1)

categorical = '4'   # categoricals

#dumb = pd.get_dummies(X_train[categorical],drop_first=True,prefix='g')
#dumby = pd.get_dummies(X_test[4],drop_first=True,prefix='j')

#X_train = X_train.drop([categorical],axis=1)
#X_test = X_test.drop([4],axis=1)

#X_train = pd.concat([X_train,dumb],axis=1)
#X_test = pd.concat([X_test,dumby],axis=1)

# feature transformation (should be applied to test data as well)

X_scaled = preprocessing.scale(X_train)

X_test_scaled = preprocessing.scale(X_test)

X_train['4'] = X_train['4'].astype('category') 

X_test[4]= X_test[4].astype('category') 


## Random Forest (gradient boosted?)

ONe thing to remember about the tree models is that you dont want to dummy-encode categoricals -> let lightgbm handle them they already have build in methods

In [ ]:


'''



mean_scores = []
stds = []
qwks_mean = []



depths = [14]

for depth in depths:
    

    params = {

    'boosting_type': 'gbdt',

    'objective': 'regression',

    'metric': {'l2', 'l1'},

    'num_leaves': 31,
        
    'max_depth': depth,

    'learning_rate': 0.05,

    'feature_fraction': 0.9,

    'bagging_fraction': 0.8,

    'bagging_freq': 5,

    'verbose': 0

    }
    
    cv = KFold(n_splits=10)

    scores = []
    qwks = []

    # if you are dealing with dataframes, you have to index with X.iloc[test_index] etc. !!1

    for train_index, test_index in cv.split(X_train,y_train):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

        lgb_train = lgb.Dataset(X_train_cv, y_train_cv)

        lgb_eval = lgb.Dataset(X_test_cv, y_test_cv, reference=lgb_train)
        
        
        gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)
        
        y_pred = gbm.predict(X_test_cv, num_iteration=gbm.best_iteration)
        
        #y_pred = np.argmax(y_pred, axis=1)

        y_pred[y_pred <= 1.12232214] = 0
        y_pred[np.where(np.logical_and(y_pred > 1.12232214, y_pred <= 1.73925866))] = 1
        y_pred[np.where(np.logical_and(y_pred > 1.73925866, y_pred <= 2.22506454))] = 2
        y_pred[y_pred > 2.22506454] = 3
        
        scores.append( mean_squared_error(y_test_cv, y_pred) ** 0.5)
        
                                 
        qwk_res = qwk(y_pred,y_test_cv)
                                 
        qwks.append(qwk_res)
        
    qwks_mean.append(np.mean(qwks))
    stds.append(np.std(scores))
    mean_scores.append(np.mean(scores))'''

In [ ]:
#plt.plot(depths,mean_scores);
#plt.xlabel("depth");plt.ylabel("cv-mean-accuracy");
#fig1, ax1 = plt.subplots();
#ax1.plot(depths,qwks_mean);
#plt.xlabel("depth")
#plt.ylabel("qwk")

In [ ]:
#qwks_mean

In [ ]:
params = {

    'boosting_type': 'gbdt',

    'objective': 'regression',

    'metric': {'l2', 'l1'},

    'num_leaves': 31,
        
    'max_depth': 14,

    'learning_rate': 0.05,

    'feature_fraction': 0.9,

    'bagging_fraction': 0.8,

    'bagging_freq': 5,

    'verbose': 0

    }
    
lgb_train = lgb.Dataset(X_train, y_train)       
        
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=2000)
        
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

y_pred[y_pred <= 1.12232214] = 0
y_pred[np.where(np.logical_and(y_pred > 1.12232214, y_pred <= 1.73925866))] = 1
y_pred[np.where(np.logical_and(y_pred > 1.73925866, y_pred <= 2.22506454))] = 2
y_pred[y_pred > 2.22506454] = 3
        
y_pred = y_pred.astype(int)

In [ ]:
d = {'installation_id': test_users, 'accuracy_group': y_pred}
sample_submission = pd.DataFrame(d)
sample_submission.to_csv('submission.csv', index=False)